In [ ]:
import numpy as np
from copy import copy
import tensorflow as tf
from importlib import reload
from random import randint, sample, shuffle
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
size = plt.rcParams["figure.figsize"]
size[0] = 2
size[1] = 2
plt.rcParams["figure.figsize"] = size

import environment as env
import visual
import trainer
import agents.random_agent as rp
import agents.Q as Q
import agents.human as human
import agents.activated as activated
import util
import agents.agent as agent
from train import train_agent
# import agents.perfect as perfect

In [ ]:
_ = reload(env)
_ = reload(Q)
_ = reload(visual)
_ = reload(activated)
_ = reload(util)
_ = reload(trainer)
# _ = reload(perfect)

## Load and Play

In [ ]:
# Create an agent
# agent1 = util.load_q_agent("agents/params/Q[16, 16]_regular.npz")
agent1 = Q.Q([16, 16])
trainer1 = train.Trainer(agent1)
# agent2 = Q.Q([16, 100, 200, 16], gamma = .6, epsilon = 0.1, beta = 0.0)
# agent2.load("4x4_server_p2_17.npz")
# Create a random playing agent
RandomAgent = rp.RandomAgentPlus()
# Create Human agent
Human = human.Human()
# Create an environment
e = env.Env(4)

In [ ]:
gui = visual.GameWithConfidences(e, agent1, Human, -1, piece_size = 100, trainer_a1 = trainer1.get_online())

In [ ]:
e.play(agent1, RandomAgent, 1000, trainer_a1 = train.Trainer(agent1, record = False).get_episode(1e-6))

In [ ]:
util.record("trials.txt", agent1, trainer1)

In [ ]:
util.new_record_file("all_trials.txt")

## Train Against Eachother

In [ ]:
# Create both agents
a1 = Q.Q([9, 100, 9], gamma = .5, epsilon = 0.1, beta = .01)
a2 = Q.Q([9, 100, 9], gamma = .5, epsilon = 0.1, beta = .01)
# Create the trainers
t1 = train.Trainer(a1)
t2 = train.Trainer(a2)
# Create the environment
train_env = env.Env(3)

In [ ]:
# Train the agents
train_env.play(a1, a2, 1000, trainer_a1 = t1.get_episode(rotate = True),
               trainer_a2 = t2.get_episode(rotate = True), final_reward = True)

In [ ]:
# Watch the agents play
gui = visual.GameWithConfidences(train_env, a1, a2, -1, piece_size = 100)

In [ ]:
# Watch winning agent play random opponent
ra = rp.RandomAgentPlus()
gui = visual.GameWithConfidences(train_env, ra, a2, -1, piece_size = 100)

In [ ]:
# Play human against winning agent
h = human.Human()
gui = visual.GameWithConfidences(train_env, a1, h, -1, piece_size = 100)

## Analyze Perfect Move Dictionaries

In [ ]:
# Get move dictionary
moves = util.get_move_dict("agents/perfect/3.txt", size = 3)

In [ ]:
_ = reload(env)
_ = reload(Q)
_ = reload(visual)
_ = reload(activated)
_ = reload(util)
_ = reload(train)
_ = reload(perfect)

In [ ]:
e = env.Env(5)
h = human.Human()
r = rp.RandomAgentPlus()
a1 = Q.Q([25, 100, 200, 500, 100, 25], gamma = .6, epsilon = 0.1, beta = 0.0)
a2 = Q.Q([25, 100, 200, 500, 100, 25], gamma = .6, epsilon = 0.1, beta = 0.0)
t1 = train.Trainer(a1, learn_rate = 1e-11, record = False, change_agent_epsilon = True, epsilon_func = lambda x: min(1.0, 10.0 / x))
t2 = train.Trainer(a2, learn_rate = 1e-11, record = False, change_agent_epsilon = True, epsilon_func = lambda x: min(1.0, 10.0 / x))
p = perfect.Perfect("agents/perfect/3.txt", 3)

In [ ]:
e.play(a1, a2, 100, trainer_a1 = t1.get_episode(), trainer_a2 = t2.get_episode())

In [ ]:
def train_perfect_move(perfect_agent, agent_trainer):
    state = util.bin_to_array(choice(list(perfect_agent.moves)))
    move = perfect_agent.get_move(state)
    reward = 10
    agent_trainer.offline([state], [move], [reward], silence = True, rotate = True)

def train_illegal_move(agent_trainer):
    state = np.int32(np.round(np.random.normal(loc = .5, scale = .01, size = [3, 3])))
    if np.sum(state) >= 9:
        state[0, 0] = 0
    # Make illegal move
    arg = choice(state.argsort(None)[-int(np.sum(state)):].tolist())
    action = np.zeros([state.size], dtype = np.int32)
    action[arg] = 1
    move = np.reshape(action, state.shape)
    reward = -100
    t.offline([state], [move], [reward], silence = True, rotate = True)

In [ ]:
n = "THIS IS THE LONGEST NAME"
a = "[000, 0000, 00000, 00000, 000]"
g = "00.00"
b = "00.00"
lr = "1e-6"
i = "1000000"
q = "+1000/1000"
s = "{name}{nsp}{arch}{asp}{gamma}{gsp}{beta}{bsp}{learn}{lrsp}{iteration}{isp}{wins}\n".format(
    name = n, arch = a, gamma = g, beta = b, learn = lr, iteration = i, wins = q, nsp = " "* (26 - len(n)),
    asp = " " * (40 - len(a)), gsp = " " * (10 - len(g)), bsp = " " * (9 - len(b)), lrsp = " " * (10 - len(lr)),
    isp = " " * (13 - len(i))
    )
print("NAME                      ARCHITECTURE                            GAMMA     BETA     L-RATE    ITERS        QUALITY")
print(("=" * 120))
print(s)